<a href="https://colab.research.google.com/github/renato-penna/fiap-tech-challenge-fase03/blob/main/fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Format Dataset

In [1]:
import json
from datasets import load_dataset # Although load_dataset is imported, we'll process line by line for memory efficiency.

DATA_PATH = "/content/drive/MyDrive/Fiap/trnTreaded.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Fiap/formatted_trn.json"

def format_dataset_into_model_input(data):
    """
    Função ajustada para receber um dicionário completo (um item do dataset)
    e extrair 'prompt' e 'completion' dele.
    """
    prompt = data.get("prompt", "")
    completion = data.get("completion", "")

    instruction = "Generate a description for the following item."

    try:
        # Extrai o texto entre "Question:" e "Answer:"
        input_text = prompt.split("Question:")[1].split("Answer:")[0].strip()
    except IndexError:
        input_text = ""

    # Extrai a resposta que vem depois de "Answer:"
    try:
        response = prompt.split("Answer:")[1].strip()
    except IndexError:
        # Se 'Answer:' não estiver no prompt, usamos o campo 'completion'
        response = completion.strip()

    return instruction, input_text, response

# Process the dataset line by line to avoid memory issues
with open(DATA_PATH, 'r', encoding='utf-8') as input_file, \
     open(OUTPUT_PATH_DATASET, 'w', encoding='utf-8') as output_file:

    for line in input_file:
        try:
            item = json.loads(line)
            instruction, input_text, response = format_dataset_into_model_input(item)

            formatted_item = {
                "instruction": instruction,
                "input": input_text,
                "output": response
            }

            output_file.write(json.dumps(formatted_item, ensure_ascii=False) + '\n')
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line.strip()} - Error: {e}")
        except Exception as e:
            print(f"An error occurred processing line: {line.strip()} - Error: {e}")

print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")

Dataset salvo em /content/drive/MyDrive/Fiap/formatted_trn.json


### Install Dependencies

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

### Setup and Imports

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments


max_seq_length = 2048
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

### Load Model and Tokenizer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

### Configure LoRA

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

### Format Prompts and Load Dataset

In [ ]:
import json
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Fiap/formatted_trn.json"


alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

### Setup SFTTrainer

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

### Train Model

In [ ]:
trainer_stats = trainer.train()

### Run Inference

In [14]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Generate a description for the following item",
        "California, The Beautiful Cookbook: Authentic Recipes from California", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a description for the following item

### Input:
California, The Beautiful Cookbook: Authentic Recipes from California

### Response:
<|end_of_text|>


### Save Model and Tokenizer

In [13]:
model.save_pretrained("/content/drive/MyDrive/Fiap/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Fiap/lora_model")

('/content/drive/MyDrive/Fiap/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Fiap/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Fiap/lora_model/tokenizer.json')